# Preliminaries

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
!pip install speechbrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 408 kB 29.5 MB/s 
     |████████████████████████████████| 86 kB 6.6 MB/s 
     |████████████████████████████████| 1.2 MB 67.1 MB/s 
     |████████████████████████████████| 596 kB 65.1 MB/s 
     |████████████████████████████████| 109 kB 73.5 MB/s 
     |████████████████████████████████| 546 kB 72.2 MB/s 
  Created wheel for hyperpyyaml: filename=HyperPyYAML-1.0.1-py3-none-any.whl size=15192 sha256=40fd985fc1657b518711b9439b9cd339b8b4216998d1be3c79f5f963e9ea0bc1
  Stored in directory: /root/.cache/pip/wheels/6c/87/65/266d722c3932f81f16332ce842e972be8421e3a9cd3771766b
Successfully built hyperpyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
!pip install pyyaml==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 636 kB 28.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [94]:
from speechbrain.pretrained import SpeakerRecognition
import torch
import torchaudio
import os
from tqdm import tqdm
import numpy as np
from sklearn.manifold import TSNE
import plotly.express as px
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pickle as pkl
import pandas as pd
from speechbrain.lobes.features import Fbank

# Loading the Verification Model

In [21]:
class SpeakerVerification(SpeakerRecognition):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.similarity = torch.nn.CosineSimilarity(dim=-1, eps=1e-6)

    @classmethod
    def from_hparams(cls, *args, **kwargs):
        verification = super(cls,cls).from_hparams(*args, **kwargs)
        source = kwargs['source']
        if os.path.exists(os.path.join(source, 'imposter_embeddings.pt')):
            verification.imp_emb = torch.load(os.path.join(source, 'imposter_embeddings.pt'))
        
        return verification
    
    def compute_snorm(self, emb1, emb2):
        emb1 = emb1.squeeze(0)
        emb2 = emb2.squeeze(0)
        score_e1 = self.similarity(emb1, self.imp_emb)
        score_e2 = self.similarity(emb1, self.imp_emb)
        score_e1_e2 = self.similarity(emb1, emb2)
        score_e1_normed = (score_e1_e2 - score_e1.mean()) / score_e1.std()
        score_e2_normed = (score_e1_e2 - score_e2.mean()) / score_e2.std()
        return score_e1_normed + score_e2_normed
          
    def verify_files(self, path_x, path_y, threshold=0.25, mean_norm=True, snorm=True):
        """Speaker verification with cosine distance
        Returns the score and the decision (0 different speakers,
        1 same speakers).
        Returns
        -------
        score
            The score associated to the binary verification output
            (cosine distance).
        prediction
            The prediction is 1 if the two signals in input are from the same
            speaker and 0 otherwise.
        """
        batch_x, _ = torchaudio.load(path_x)
        batch_y, _ = torchaudio.load(path_y)
        # Verify:
        emb1 = self.encode_batch(batch_x, normalize=mean_norm)
        emb2 = self.encode_batch(batch_y, normalize=mean_norm)
        # SNorm
        if snorm and hasattr(self, 'imp_emb'):
            score = self.compute_snorm(emb1, emb2)
        else:
            score = self.similarity(emb1, emb2)
        decision = score > threshold
        # Squeeze:
        return score[0], decision[0]
    def get_output_embeddings(self, path_x, path_y, threshold=0.25, mean_norm=True, snorm=True):
        batch_x, _ = torchaudio.load(path_x)
        batch_y, _ = torchaudio.load(path_y)
        # Verify:
        emb1 = self.encode_batch(batch_x, normalize=mean_norm)
        emb2 = self.encode_batch(batch_y, normalize=mean_norm)
        return emb1, emb2
    def get_speaker_vector(self, path_x, mean_norm=True, snorm=True):
        batch_x, _ = torchaudio.load(path_x)
        embedding = self.encode_batch(batch_x, normalize=mean_norm)[0][0].numpy()
        return embedding
    def get_input_vector(self, path_x, mean_norm=True, snorm=True):
        batch_x, _ = torchaudio.load(path_x)
        return batch_x.numpy()[0]

In [7]:
!rm -rf /content/temp_verification_model/
!mkdir /content/temp_verification_model/
!cp -r /gdrive/MyDrive/shared_space/ecapa-tdnn/* /content/temp_verification_model/

In [22]:
# Note: This needs a GPU!
verification_model = SpeakerVerification.from_hparams(source="/content/temp_verification_model/", hparams_file='hparams_inference.yaml', savedir="/tmp")

In [23]:
verification_model.get_input_vector('/content/cv/content/speakers/male-21/33.wav').shape

(40704,)

In [ ]:
# score, pred = verification_model.verify_files(file_path1, file_path2, threshold=10)

# Loading the CommonVoice Sample

In [9]:
!rm -rf /content/cv/
!mkdir /content/cv/

In [10]:
!unzip /gdrive/MyDrive/shared_space/commonvoice-farsi/cv-sample-speaker-dependent.zip -d /content/cv/

Archive:  /gdrive/MyDrive/shared_space/commonvoice-farsi/cv-sample-speaker-dependent.zip
   creating: /content/cv/content/speakers/
   creating: /content/cv/content/speakers/female-0/
  inflating: /content/cv/content/speakers/female-0/18.wav  
  inflating: /content/cv/content/speakers/female-0/2.wav  
  inflating: /content/cv/content/speakers/female-0/37.wav  
  inflating: /content/cv/content/speakers/female-0/14.wav  
  inflating: /content/cv/content/speakers/female-0/36.wav  
  inflating: /content/cv/content/speakers/female-0/46.wav  
  inflating: /content/cv/content/speakers/female-0/49.wav  
  inflating: /content/cv/content/speakers/female-0/6.wav  
  inflating: /content/cv/content/speakers/female-0/0.wav  
  inflating: /content/cv/content/speakers/female-0/43.wav  
  inflating: /content/cv/content/speakers/female-0/13.wav  
  inflating: /content/cv/content/speakers/female-0/27.wav  
  inflating: /content/cv/content/speakers/female-0/7.wav  
  inflating: /content/cv/content/speaker

In [101]:
feature_maker = Fbank()
def get_fbank(file_path):
  np_arr = verification_model.get_input_vector(file_path).reshape(1,-1)
  return feature_maker(torch.Tensor(np_arr)).numpy()[0]

# Getting the inputs and outputs 

In [12]:
# vectors = []
# labels = []
# for gender in ['male','female']:
#   for speaker_num in range(25):
#     for file_num in range(50):
#       file_path = f'/content/cv/content/speakers/{gender}-{speaker_num}/{file_num}.wav'
#       curr_out = verification_model.get_speaker_vector(file_path)
#       vectors.append(curr_out)
#       labels.append(f'{gender}-{speaker_num}')

In [119]:
fbanks = []
for gender in ['male','female']:
  for speaker_num in range(25):
    for file_num in range(50):
      file_path = f'/content/cv/content/speakers/{gender}-{speaker_num}/{file_num}.wav'
      curr_fbank = get_fbank(file_path)
      fbanks.append(curr_fbank)

In [116]:
fbanks.reshape(2500,20*40).shape

(2500, 800)

In [120]:
fbanks = np.array([item[:20] for item in fbanks])
fbanks = fbanks.reshape(2500,20*40)
tsne_fbanks = TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(fbanks)
tsne_fbanks_2d = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(fbanks)
pca_fbanks = PCA(n_components=3, svd_solver='full').fit_transform(fbanks)
pca_fbanks_2d = PCA(n_components=2, svd_solver='full').fit_transform(fbanks)

In [31]:
# inputs = []
# for gender in ['male','female']:
#   for speaker_num in range(25):
#     for file_num in range(50):
#       file_path = f'/content/cv/content/speakers/{gender}-{speaker_num}/{file_num}.wav'
#       curr_in = verification_model.get_input_vector(file_path)
#       inputs.append(curr_in)

In [140]:
with open('/gdrive/MyDrive/arman/cv-visualization/vectors.pkl', 'rb') as file:
  vectors = pkl.load(file)
with open('/gdrive/MyDrive/arman/cv-visualization/vectors-tsne-3d.pkl', 'rb') as file:
  tsne_vectors = pkl.load(file)
with open('/gdrive/MyDrive/arman/cv-visualization/vectors-tsne-2d.pkl', 'rb') as file:
  tsne_vectors_2d = pkl.load(file)
with open('/gdrive/MyDrive/arman/cv-visualization/labels.pkl', 'rb') as file:
  labels = pkl.load(file)
with open('/gdrive/MyDrive/arman/cv-visualization/inputs.pkl', 'rb') as file:
  inputs = pkl.load(file)
with open('/gdrive/MyDrive/arman/cv-visualization/inputs-tsne-3d.pkl', 'rb') as file:
  tsne_inputs = pkl.load(file)
with open('/gdrive/MyDrive/arman/cv-visualization/inputs-tsne-2d.pkl', 'rb') as file:
  tsne_inputs_2d = pkl.load(file)
with open('/gdrive/MyDrive/arman/cv-visualization/vectors-pca-3d.pkl', 'rb') as file:
  pca_vectors = pkl.load(file)
with open('/gdrive/MyDrive/arman/cv-visualization/vectors-pca-2d.pkl', 'rb') as file:
  pca_vectors_2d = pkl.load(file)
with open('/gdrive/MyDrive/arman/cv-visualization/inputs-pca-3d.pkl', 'rb') as file:
  pca_inputs = pkl.load(file)
with open('/gdrive/MyDrive/arman/cv-visualization/inputs-pca-2d.pkl', 'rb') as file:
  pca_inputs_2d = pkl.load(file)
with open('/gdrive/MyDrive/arman/cv-visualization/fbanks-pca-2d.pkl', 'rb') as file:
  pca_fbanks_2d = pkl.load(file)
with open('/gdrive/MyDrive/arman/cv-visualization/fbanks-pca-3d.pkl', 'rb') as file:
  pca_fbanks = pkl.load(file)
with open('/gdrive/MyDrive/arman/cv-visualization/fbanks-tsne-2d.pkl', 'rb') as file:
  tsne_fbanks_2d = pkl.load(file)
with open('/gdrive/MyDrive/arman/cv-visualization/fbanks-tsne-3d.pkl', 'rb') as file:
  tsne_fbanks = pkl.load(file)

In [32]:
# vectors = np.array(vectors)
# inputs = np.array([item[:16000] for item in inputs])
# tsne_inputs = TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(inputs)
# tsne_vectors_2d = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(vectors)
# tsne_inputs_2d = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(inputs)
# tsne_vectors = TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(vectors)
# pca_inputs = PCA(n_components=3, svd_solver='full').fit_transform(inputs)
# pca_inputs_2d = PCA(n_components=2, svd_solver='full').fit_transform(inputs)
# pca_vectors = PCA(n_components=3, svd_solver='full').fit_transform(vectors)
# pca_vectors_2d = PCA(n_components=2, svd_solver='full').fit_transform(vectors)

(2500, 16000)

In [125]:
# with open('/gdrive/MyDrive/arman/vectors.pkl', 'wb') as file:
#   pkl.dump(vectors, file)
# with open('/gdrive/MyDrive/arman/vectors-tsne-3d.pkl', 'wb') as file:
#   pkl.dump(vectors, file)
# with open('/gdrive/MyDrive/arman/labels.pkl', 'wb') as file:
#   pkl.dump(labels, file)
# with open('/gdrive/MyDrive/arman/inputs.pkl', 'wb') as file:
#   pkl.dump(inputs, file)
# with open('/gdrive/MyDrive/arman/inputs-tsne-3d.pkl', 'wb') as file:
#   pkl.dump(tsne_inputs, file)
# with open('/gdrive/MyDrive/arman/inputs-tsne-2d.pkl', 'wb') as file:
#   pkl.dump(tsne_inputs_2d, file)
# with open('/gdrive/MyDrive/arman/vectors-tsne-2d.pkl', 'wb') as file:
#   pkl.dump(tsne_vectors_2d, file)
# with open('/gdrive/MyDrive/arman/inputs-pca-3d.pkl', 'wb') as file:
#   pkl.dump(pca_inputs, file)
# with open('/gdrive/MyDrive/arman/inputs-pca-2d.pkl', 'wb') as file:
#   pkl.dump(pca_inputs_2d, file)
# with open('/gdrive/MyDrive/arman/vectors-pca-3d.pkl', 'wb') as file:
#   pkl.dump(pca_vectors, file)
# with open('/gdrive/MyDrive/arman/vectors-pca-2d.pkl', 'wb') as file:
#   pkl.dump(pca_vectors_2d, file)
# with open('/gdrive/MyDrive/arman/cv-visualization/fbanks.pkl', 'wb') as file:
#   pkl.dump(fbanks, file)
# with open('/gdrive/MyDrive/arman/cv-visualization/fbanks-pca-2d.pkl', 'wb') as file:
#   pkl.dump(pca_fbanks_2d, file)
# with open('/gdrive/MyDrive/arman/cv-visualization/fbanks-pca-3d.pkl', 'wb') as file:
#   pkl.dump(pca_fbanks, file)
# with open('/gdrive/MyDrive/arman/cv-visualization/fbanks-tsne-2d.pkl', 'wb') as file:
#   pkl.dump(tsne_fbanks_2d, file)
# with open('/gdrive/MyDrive/arman/cv-visualization/fbanks-tsne-3d.pkl', 'wb') as file:
#   pkl.dump(tsne_fbanks, file)

In [ ]:
# fbanks = np.array([item[:20] for item in fbanks])
# fbanks = fbanks.reshape(2500,20*40)
# tsne_fbanks = TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(fbanks)
# tsne_fbanks_2d = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(fbanks)
# pca_fbanks = PCA(n_components=3, svd_solver='full').fit_transform(fbanks)
# pca_fbanks_2d = PCA(n_components=2, svd_solver='full').fit_transform(fbanks)

# Visualization (CommonVoice)

## Inputs - 3D - TSNE

In [82]:
df = pd.DataFrame({'dim1': tsne_inputs[:, 0], 'dim2': tsne_inputs[:, 1], 'dim3': tsne_inputs[:, 2]})
df['label'] = labels
df.head()
fig = px.scatter_3d(df, x = 'dim1',
                    y = 'dim2',
                    z = 'dim3',
                    color = 'label') 
fig.show()

## Outputs - 3D - TSNE

In [83]:
df = pd.DataFrame({'dim1': tsne_vectors[:, 0], 'dim2': tsne_vectors[:, 1], 'dim3': tsne_vectors[:, 2]})
df['label'] = labels
df.head()

,dim1,dim2,dim3,label
0,3.017386,-9.301327,5.712929,male-0
1,-11.598872,2.975524,-8.082460,male-0
2,-12.732428,-2.958466,5.083163,male-0
3,0.628090,0.784373,-1.311230,male-0
4,-4.000675,-18.349646,-3.934532,male-0


In [84]:
fig = px.scatter_3d(df, x = 'dim1',
                    y = 'dim2',
                    z = 'dim3',
                    color = 'label') 
fig.show()

## FBanks - 3D - TSNE

In [121]:
df = pd.DataFrame({'dim1': tsne_fbanks[:, 0], 'dim2': tsne_fbanks[:, 1], 'dim3': tsne_fbanks[:, 2]})
df['label'] = labels
fig = px.scatter_3d(df, x = 'dim1',
                    y = 'dim2',
                    z = 'dim3',
                    color = 'label') 
fig.show()

## Inputs - TSNE - 2D

In [47]:
df = pd.DataFrame({'dim1': tsne_inputs_2d[:, 0], 'dim2': tsne_inputs_2d[:, 1]})
df['label'] = labels
fig = px.scatter(df, x = 'dim1',
                    y = 'dim2',
                    color = 'label') 
fig.show()

## FBanks - TSNE - 2D

In [141]:
df = pd.DataFrame({'dim1': tsne_fbanks_2d[:, 0], 'dim2': tsne_fbanks_2d[:, 1]})
df['label'] = labels
fig = px.scatter(df, x = 'dim1',
                    y = 'dim2',
                    color = 'label') 
fig.show()

Outputs - TSNE - 2D

In [48]:
df = pd.DataFrame({'dim1': tsne_vectors_2d[:, 0], 'dim2': tsne_vectors_2d[:, 1]})
df['label'] = labels
fig = px.scatter(df, x = 'dim1',
                    y = 'dim2',
                    color = 'label') 
fig.show()

FBanks - TSNE - 2D

In [124]:
df = pd.DataFrame({'dim1': tsne_fbanks_2d[:, 0], 'dim2': tsne_fbanks_2d[:, 1]})
df['label'] = labels
fig = px.scatter(df, x = 'dim1',
                    y = 'dim2',
                    color = 'label') 
fig.show()

## Inputs - PCA - 3D

In [57]:
df = pd.DataFrame({'dim1': pca_inputs[:, 0], 'dim2': pca_inputs[:, 1], 'dim3': pca_inputs[:, 2]})
df['label'] = labels
df.head()
fig = px.scatter_3d(df, x = 'dim1',
                    y = 'dim2',
                    z = 'dim3',
                    color = 'label') 
fig.show()

## Outputs - PCA - 3D

In [86]:
df = pd.DataFrame({'dim1': pca_vectors[:, 0], 'dim2': pca_vectors[:, 1], 'dim3': pca_vectors[:, 2]})
df['label'] = labels
df.head()
fig = px.scatter_3d(df, x = 'dim1',
                    y = 'dim2',
                    z = 'dim3',
                    color = 'label') 
fig.show()

## Fbanks - PCA - 3D

In [122]:
df = pd.DataFrame({'dim1': pca_fbanks[:, 0], 'dim2': pca_fbanks[:, 1], 'dim3': pca_fbanks[:, 2]})
df['label'] = labels
df.head()
fig = px.scatter_3d(df, x = 'dim1',
                    y = 'dim2',
                    z = 'dim3',
                    color = 'label') 
fig.show()

## Inputs - PCA - 2D

In [60]:
df = pd.DataFrame({'dim1': pca_inputs_2d[:, 0], 'dim2': pca_inputs_2d[:, 1]})
df['label'] = labels
fig = px.scatter(df, x = 'dim1',
                    y = 'dim2',
                    color = 'label') 
fig.show()

## Outputs - PCA - 2D

In [87]:
df = pd.DataFrame({'dim1': pca_vectors_2d[:, 0], 'dim2': pca_vectors_2d[:, 1]})
df['label'] = labels
fig = px.scatter(df, x = 'dim1',
                    y = 'dim2',
                    color = 'label') 
fig.show()

## FBanks - PCA - 2D

In [123]:
df = pd.DataFrame({'dim1': pca_fbanks_2d[:, 0], 'dim2': pca_fbanks_2d[:, 1]})
df['label'] = labels
fig = px.scatter(df, x = 'dim1',
                    y = 'dim2',
                    color = 'label') 
fig.show()

# Visualization (Our Samples) (Car Commands)

In [67]:
list_of_file_names = [str(i)+'.wav' for i in range(1,26)]
list_of_file_names.extend(['26-1.wav','26-2.wav','26-3.wav','26-4.wav','26-5.wav'])
list_of_file_names.extend([str(i)+'.wav' for i in range(27, 41)])

In [66]:
list_of_speakers = ['amirinezhad','chavoshi','dehghanmonfared','dehghanpoor','hosseini','jafarpisheh',
                    'maghsoodloo','malekzadeh','ramezani','razavi','shayanfar']
list_of_speakers.extend(['woman'+str(i) for i in range(1, 10)])

In [134]:
our_path = '/gdrive/MyDrive/shared_space/car-commands/'
vectors = []
labels = []
inputs = []
for speaker in list_of_speakers:
  for file_name in list_of_file_names:
    file_path = f'{our_path}/{speaker}/{file_name}'
    curr_out = verification_model.get_speaker_vector(file_path)
    curr_in = verification_model.get_input_vector(file_path)
    vectors.append(curr_out)
    inputs.append(curr_in)
    labels.append(speaker)

In [127]:
our_path = '/gdrive/MyDrive/shared_space/car-commands/'
fbanks = []
for speaker in list_of_speakers:
  for file_name in list_of_file_names:
    file_path = f'{our_path}/{speaker}/{file_name}'
    curr_fbank = get_fbank(file_path)
    fbanks.append(curr_fbank)

In [129]:
fbanks = np.array([item[:20] for item in fbanks])
fbanks = fbanks.reshape(880,20*40)
tsne_fbanks = TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(fbanks)
tsne_fbanks_2d = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(fbanks)
pca_fbanks = PCA(n_components=3, svd_solver='full').fit_transform(fbanks)
pca_fbanks_2d = PCA(n_components=2, svd_solver='full').fit_transform(fbanks)

In [130]:
# with open('/gdrive/MyDrive/arman/our-visualization/fbanks.pkl', 'wb') as file:
#   pkl.dump(fbanks, file)
# with open('/gdrive/MyDrive/arman/our-visualization/fbanks-pca-2d.pkl', 'wb') as file:
#   pkl.dump(pca_fbanks_2d, file)
# with open('/gdrive/MyDrive/arman/our-visualization/fbanks-pca-3d.pkl', 'wb') as file:
#   pkl.dump(pca_fbanks, file)
# with open('/gdrive/MyDrive/arman/our-visualization/fbanks-tsne-2d.pkl', 'wb') as file:
#   pkl.dump(tsne_fbanks_2d, file)
# with open('/gdrive/MyDrive/arman/our-visualization/fbanks-tsne-3d.pkl', 'wb') as file:
#   pkl.dump(tsne_fbanks, file)

In [69]:
vectors = np.array(vectors)
inputs = np.array([item[:16000] for item in inputs])
tsne_inputs = TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(inputs)
tsne_vectors_2d = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(vectors)
tsne_inputs_2d = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(inputs)
tsne_vectors = TSNE(n_components=3, learning_rate='auto', init='random').fit_transform(vectors)
pca_inputs = PCA(n_components=3, svd_solver='full').fit_transform(inputs)
pca_inputs_2d = PCA(n_components=2, svd_solver='full').fit_transform(inputs)
pca_vectors = PCA(n_components=3, svd_solver='full').fit_transform(vectors)
pca_vectors_2d = PCA(n_components=2, svd_solver='full').fit_transform(vectors)

In [79]:
# !mkdir /gdrive/MyDrive/arman/our-visualization/

In [135]:
# with open('/gdrive/MyDrive/arman/our-visualization/vectors.pkl', 'wb') as file:
#   pkl.dump(vectors, file)
# with open('/gdrive/MyDrive/arman/our-visualization/vectors-tsne-3d.pkl', 'wb') as file:
#   pkl.dump(vectors, file)
# with open('/gdrive/MyDrive/arman/our-visualization/labels.pkl', 'wb') as file:
#   pkl.dump(labels, file)
# with open('/gdrive/MyDrive/arman/our-visualization/inputs.pkl', 'wb') as file:
#   pkl.dump(inputs, file)
# with open('/gdrive/MyDrive/arman/our-visualization/inputs-tsne-3d.pkl', 'wb') as file:
#   pkl.dump(tsne_inputs, file)
# with open('/gdrive/MyDrive/arman/our-visualization/inputs-tsne-2d.pkl', 'wb') as file:
#   pkl.dump(tsne_inputs_2d, file)
# with open('/gdrive/MyDrive/arman/our-visualization/vectors-tsne-2d.pkl', 'wb') as file:
#   pkl.dump(tsne_vectors_2d, file)
# with open('/gdrive/MyDrive/arman/our-visualization/inputs-pca-3d.pkl', 'wb') as file:
#   pkl.dump(pca_inputs, file)
# with open('/gdrive/MyDrive/arman/our-visualization/inputs-pca-2d.pkl', 'wb') as file:
#   pkl.dump(pca_inputs_2d, file)
# with open('/gdrive/MyDrive/arman/our-visualization/vectors-pca-3d.pkl', 'wb') as file:
#   pkl.dump(pca_vectors, file)
# with open('/gdrive/MyDrive/arman/our-visualization/vectors-pca-2d.pkl', 'wb') as file:
#   pkl.dump(pca_vectors_2d, file)

## Inputs - 3D - TSNE

In [70]:
df = pd.DataFrame({'dim1': tsne_inputs[:, 0], 'dim2': tsne_inputs[:, 1], 'dim3': tsne_inputs[:, 2]})
df['label'] = labels
df.head()
fig = px.scatter_3d(df, x = 'dim1',
                    y = 'dim2',
                    z = 'dim3',
                    color = 'label') 
fig.show()

## Outputs - 3D - TSNE

In [71]:
df = pd.DataFrame({'dim1': tsne_vectors[:, 0], 'dim2': tsne_vectors[:, 1], 'dim3': tsne_vectors[:, 2]})
df['label'] = labels
df.head()

,dim1,dim2,dim3,label
0,8.712044,12.043658,-1.275852,amirinezhad
1,9.833485,10.920026,-0.238112,amirinezhad
2,9.815046,11.499156,-0.475027,amirinezhad
3,8.876072,12.153971,-0.320188,amirinezhad
4,9.736080,12.178397,0.968589,amirinezhad


In [72]:
fig = px.scatter_3d(df, x = 'dim1',
                    y = 'dim2',
                    z = 'dim3',
                    color = 'label') 
fig.show()

## FBanks - 3D - TSNE

In [136]:
df = pd.DataFrame({'dim1': tsne_fbanks[:, 0], 'dim2': tsne_fbanks[:, 1], 'dim3': tsne_fbanks[:, 2]})
df['label'] = labels
fig = px.scatter_3d(df, x = 'dim1',
                    y = 'dim2',
                    z = 'dim3',
                    color = 'label') 
fig.show()

## Inputs - TSNE - 2D

In [73]:
df = pd.DataFrame({'dim1': tsne_inputs_2d[:, 0], 'dim2': tsne_inputs_2d[:, 1]})
df['label'] = labels
fig = px.scatter(df, x = 'dim1',
                    y = 'dim2',
                    color = 'label') 
fig.show()

## FBanks - TSNE - 2D

In [139]:
df = pd.DataFrame({'dim1': tsne_fbanks_2d[:, 0], 'dim2': tsne_fbanks_2d[:, 1]})
df['label'] = labels
fig = px.scatter(df, x = 'dim1',
                    y = 'dim2',
                    color = 'label') 
fig.show()

Outputs - TSNE - 2D

In [74]:
df = pd.DataFrame({'dim1': tsne_vectors_2d[:, 0], 'dim2': tsne_vectors_2d[:, 1]})
df['label'] = labels
fig = px.scatter(df, x = 'dim1',
                    y = 'dim2',
                    color = 'label') 
fig.show()

## Inputs - PCA - 3D

In [75]:
df = pd.DataFrame({'dim1': pca_inputs[:, 0], 'dim2': pca_inputs[:, 1], 'dim3': pca_inputs[:, 2]})
df['label'] = labels
df.head()
fig = px.scatter_3d(df, x = 'dim1',
                    y = 'dim2',
                    z = 'dim3',
                    color = 'label') 
fig.show()

## Outputs - PCA - 3D

In [76]:
df = pd.DataFrame({'dim1': pca_vectors[:, 0], 'dim2': pca_vectors[:, 1], 'dim3': pca_vectors[:, 2]})
df['label'] = labels
df.head()
fig = px.scatter_3d(df, x = 'dim1',
                    y = 'dim2',
                    z = 'dim3',
                    color = 'label') 
fig.show()

## Fbanks - PCA - 3D

In [137]:
df = pd.DataFrame({'dim1': pca_fbanks[:, 0], 'dim2': pca_fbanks[:, 1], 'dim3': pca_fbanks[:, 2]})
df['label'] = labels
df.head()
fig = px.scatter_3d(df, x = 'dim1',
                    y = 'dim2',
                    z = 'dim3',
                    color = 'label') 
fig.show()

## Inputs - PCA - 2D

In [77]:
df = pd.DataFrame({'dim1': pca_inputs_2d[:, 0], 'dim2': pca_inputs_2d[:, 1]})
df['label'] = labels
fig = px.scatter(df, x = 'dim1',
                    y = 'dim2',
                    color = 'label') 
fig.show()

## Outputs - PCA - 2D

In [78]:
df = pd.DataFrame({'dim1': pca_vectors_2d[:, 0], 'dim2': pca_vectors_2d[:, 1]})
df['label'] = labels
fig = px.scatter(df, x = 'dim1',
                    y = 'dim2',
                    color = 'label') 
fig.show()

## FBanks - PCA - 2D

In [138]:
df = pd.DataFrame({'dim1': pca_fbanks_2d[:, 0], 'dim2': pca_fbanks_2d[:, 1]})
df['label'] = labels
fig = px.scatter(df, x = 'dim1',
                    y = 'dim2',
                    color = 'label') 
fig.show()